实现卷积神经网络


In [8]:
#coding:utf-8
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
mnist=input_data.read_data_sets('mnist_data',one_hot=True)
#one_hot 独热码的编码（encoding）形式
#0，1，2，3，4，5，6，7，8，9的十位数字


Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting mnist_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [11]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x11f932eb8>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x11f932cc0>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x11f932d68>)

下面构建测试数据


In [19]:
#None 表示张量（Tensor）的第一个维度可以是任何长度
input_x=tf.placeholder(tf.float32,[None,28*28])/255#是灰度值
output_y=tf.placeholder(tf.int32,[None,10])#10个数字的标签
input_x_images=tf.reshape(input_x,[-1,28,28,1])#改变形状之后的输入
#从测试数据集里选取3000个手写数字的图片和对应的标签
test_x=mnist.test.images[:3000]#图片
test_y=mnist.test.labels[:3000]#标签
#构建卷积神经网络


开始构建卷积神经网络


In [40]:
#第一层卷积

conv1=tf.layers.conv2d(
    inputs=input_x_images,#形状是[28,28,1]
    filters=32, #32个过滤器，输出的深度(depth)32
    kernel_size=[5,5], #过滤器在二维的大小是(5*5)
    strides=1,#步长是1
    padding='same', #输出也是28*28
    activation=tf.nn.relu#激活函数是Relu
    #形状是28*28*32
)
#第一层池化(亚采样)
pool1=tf.layers.max_pooling2d(
    inputs=conv1,#形状是[28,28,32]
    pool_size=[2,2],#过滤器在二维的大小是(2*2)
    strides=2,#步长是2
)#形状[14,14,32]
#定义第二层卷积
conv2=tf.layers.conv2d(
    inputs=pool1,#形状是[14,14,32]
    filters=64,  #32个过滤器，输出的深度(depth)64
    kernel_size=[5,5], #过滤器在二维的大小是(5*5)
    strides=1,#步长是1
    padding='same', #输出也是28*28
    activation=tf.nn.relu#激活函数是Relu
    #形状是14*14*64
)
#第二层池化(亚采样)
pool2=tf.layers.max_pooling2d(
    inputs=conv2,#形状是[14,14,64]
    pool_size=[2,2],#过滤器在二维的大小是(2*2)
    strides=2,#步长是2
)#形状[7,7,64]
#平坦化(flat)
flat=tf.reshape(pool2,[-1,7*7*64])#形状[7*7*64]
#1024个神经元的全连接层
dense=tf.layers.dense(inputs=flat,units=1024,activation=tf.nn.relu)
#加入dropout防止过拟合
dropout=tf.layers.dropout(inputs=dense,rate=0.5)
#构建10个神经元的全连接层，这里不用激活函数来做非线性化了
logits=tf.layers.dense(inputs=dropout,units=10)#输出形状[1*1*10]
#计算误差（计算Cross entropy(交叉熵)，再用Softmax计算出百分比的概率）
loss=tf.losses.softmax_cross_entropy(onehot_labels=output_y,logits=logits)
#再用优化器Adam来进行优化，最小化误差。学习率0.001
train_op=tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
#计算神经网络的精度。计算预测值和实际标签值得匹配程度
#返回(accuracy,update_op)创建两个局部变量
accuracy=tf.metrics.accuracy(
        labels=tf.argmax(output_y,axis=1),
        predictions=tf.argmax(logits,axis=1))[1]
#创建回话
sess=tf.Session()
#初始化变量:全局和局部
init=tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
sess.run(init)



开始训练构建好的神经网络


In [42]:
for _ in range(20000):
    batch=mnist.train.next_batch(50)#从Train训练数据集取下一个50个样本
    train_loss,train_op_=sess.run([loss,train_op],{input_x:batch[0],output_y:batch[1]})
    if _%100==0:
        test_accuracy=sess.run(accuracy,{input_x:test_x,output_y:test_y})
        print("Step=%d, Train loss=%.4f,[Test accuracy=%.2f]"%(_,train_loss,test_accuracy))
#打印20个预测值和真实值得对
test_output=sess.run(logits,{input_x:test_x[:20]})
inferenced_y=np.argmax(test_output,1)
print(inferenced_y,"Inference numbers")#推测的数字
print(np.argmax(test_y[:20],1),'Real number')#真实的数字
    

Step=0, Train loss=0.0391,[Test accuracy=0.79]


KeyboardInterrupt: 